# Exploratory data analysis with BayesDB

Authored by: [Ulrich Schaechtle](www.schaechtle.com) of the MIT Probabilistic
Computing Project (Probcomp). Prepared for in February, 2018. This is a
generalization of the gapminder-exploratory analysis, authored by
[Feras Saad](http://fsaad.mit.edu).

This notebook,will serve as a template for  exploratory date analysis with
BayesDB. It should work on any analysis-ready `.csv`-file.

A user needs to change.

This notebook will cover the following topics:

- Activating `jupyter_probcomp` magics and associated libraries.
- Customizing this notebook for your own data!
- Defining experimental outcomes of interest.
- Creating a BayesDB file on disk, which will store data and models.
- Ingesting data from .csv files into BayesDB.
- Basic data manipulation, subsampling and plotting using SQL and
`jupyter_probcomp` magics.
- Creating populations for database tables using the Metamodeling Language
(MML).
- Producing visualizations Cross-Categorization (CrossCat), the default model
discovery method for populations in BayesDB.
- Building an ensemble of CrossCat models, and visualizing their aggregate
properties.
- Using the Bayesian Query Language (BQL) to query the ensemble of CrossCat
models for exploratory tasks, such as (i) detecting variables which are probably
dependent, and (ii) finding database records which are probably predictive of
one another.

### Setting up the Jupyter environment

The first step is to load the `jupyter_probcomp.magics` library, which provides
BayesDB hooks for data exploration, plotting, querying, and analysis through
this Jupyter notebook environment. The second cell allows plots from matplotlib
and javascript to be shown inline.

In [ ]:
%load_ext jupyter_probcomp.magics

In [ ]:
%matplotlib inline
%vizgpm inline

## Customize this notebook for your data!

Put a path to an analysis ready `.csv` here. If you are not sure how such a .csv
file should be formatted -- please read the [population assembly
tutorial](../population-assembly-tutorial.ipynb).

In [ ]:
csv_file_path = 'your-file.csv'

### Defining experimental outcomes of interests
Define which variables/columns in your data tables are of particular interest.
Those will henceforth be called `outcomes`. Ensure that the strings contain
double qouted names:

In [ ]:
outcomes = [
    '"x"',
    '"y"',
]

### Creating a BayesDB `.bdb` file on disk

First, we remove all files with the name of the current .bdb file to avoid confusion. Make sure to either rename the `.bdb` file below or comment out the next line for further analysis.

In [ ]:
!rm -f bayesian_database.bdb

We next use the `%bayesdb` magic to create a `.bdb` file on disk named
`bayesian_database.bdb`. This file will store all the data and models created in this
session.

In [ ]:
%bayesdb bayesian_database.bdb

### Ingesting data from a `.csv` file into a BayesDB table

All datasets that are considered analysis ready are stored in form of csv files.

In [ ]:
%bql CREATE TABLE "data_full" FROM '{csv_file_path}'

Due to environmnt variables set in SQL-lite, we need to subsample to  ensure we have less than 1000 columns. If the cell above throw an error indicating too many columns then turn the two cells below into code cells (using the drop down menu in above in the menu bar.

Each column of the csv file is a variable, and each row is a record. We use the `CREATE TABLE` BQL query, with the pathname of the csv file, to convert the csv data into a database table named `data_full`.

Almost all datasets have missing values, and special tokens such as `NaN` or
`NA` indicating a particular cell is missing. In most data, empty
strings are used. To tell BayesDB to treat empty strings as SQL `NULL` we use
the `.nullify` command, followed by the name of the table and the string `''`
which represents missing data.

In [ ]:
%bql .nullify data_full ''

We further repeat the exercise for the strings `NA` and `NaN`.

In [ ]:
%bql .nullify data_full 'NA'
%bql .nullify data_full 'NaN'

#### If users know about any other missing values, they should edit the cells above


### Running basic queries on the table using BQL and SQL

Now that the dataset has been loaded into at table, and missing values
converted to `NULL`, we can run standard SQL queries to explore the contents of
the data. For example, we can select the first 5 records.

In [ ]:
%bql SELECT * FROM "data_full" LIMIT 5;

We can also find the total number of records.

In [ ]:
%bql SELECT COUNT(*) FROM "data_full";

### Downsampling the columns in the table

The full data table (`data_full`) may contains a large number of columns.
In this notebook, our exploratory analysis will be based on a random subsample of
100 columns. To create the subsample, we use the `.subsample_columns` magic. The
`--keep` flag accepts a list of column names which should be kept. We will keep 
the columns denoted as experimental outcomes, since it is the identifier for
each record. The `--seed` flag specifies
the random seed to create the subsample, which will ensure our analyses are
reproducible. Finally, `data_full` is the original table, `data` is
the name of the new table, and 100 is the number of columns to downsample to.

In [ ]:
outcomes_str = ' '.join(outcomes)
%bql .subsample_columns --seed=8 data_full data_column_subsampled 100 --keep  {outcomes_str}

### Downsampling the rows in the table

We further may wish to downsample the number of rows.

In [ ]:
%%bql
CREATE TABLE data AS SELECT * FROM data_column_subsampled 
    ORDER BY bql_rand() LIMIT 100

### Creating a BayesDB population for data

The notion of a "population" is a central concept in BayesDB. For a standard
database table, such as `data`, each column is associated with a [data
type](https://sqlite.org/datatype3.html), which in sqlite3 are `TEXT`, `REAL`,
`INTEGER`, and `BLOB`. For a BayesDB population, each variable is associated
with a _statistical data type_. 

These statistical types, such as `NOMINAL`,
`NUMERICAL`, `MAGNITUDE`, and `COUNTS`, specify the set of values and default
probability distributions used for building probabilistic models of the data in
the population. In this tutorial, we will use the `NUMERICAL` and `NOMINAL`
statistical data types.

We can use the `GUESS SCHEMA FOR <table>` command from the Metamodeling Language
(MML) in BayesDB to guess the statistical data types of variables in the table.
The guesses use heuristics based on the contents in the cells. The
`num_distinct` column shows the number of unique values for that variable, and
the `reason` column explains which heuristic was used to make the guess.

In [ ]:
%mml GUESS SCHEMA FOR "data"

We use this case to create a population for the `data` table. The population
schema uses the statistical types guessed by BayesDB (from the previous cell) for all
variables.

In [ ]:
%%mml
CREATE POPULATION FOR "data" WITH SCHEMA (
    -- Use the guesses from the previous cell for all variables.
    GUESS STATTYPES OF (*);
);

### Visualizing joint distributions of data in the population

Equipped with the statistical data types of variables in the population, we can
now use the plotting features of BayesDB to produce scatter plots and heatmaps
for the marginal and (pairwise) joint distributions of variables of interest.
The `.interactive_pairplot` command requires a flag `--population=<pop>` for the
population name, followed a BQL query. It generates pairplots of the data in all
pairs of columns yielded by the query . Below, we have selected the experimental
outcomes:

In [ ]:
outcomes_str = ','.join(outcomes)
%bql .interactive_pairplot --population=data SELECT {outcomes_str} FROM data_full

### Creating a generator for the population using CrossCat

Now that we have created the `data` population, the next step is to analyze
the data by building probabilistic models which explain the data generating
process. Probabilistic data analyses in BayesDB are specified by declaring
`GENERATOR` for a population. The default generator in BayesDB is based on
Cross-Categorization [(Crosscat)](http://jmlr.org/papers/v17/11-392.html). The
CrossCat generator is a Bayesian factorial mixture model which learns a full
joint distribution over all variables in the population, using a divide-and-
conquer approach. We will explore CrossCat more in this notebook.

For now we use MML to declare the a generator for the `data` population.
Note that that we have left the schema (there are not specific model commands or
overrides), which will apply the built-in default model discovery strategies.

In [ ]:
%mml CREATE GENERATOR FOR "data";

After creating the generator, we now need to initialize `MODELS` for the
generator. We can think of a `GENERATOR` as specifying a hypothesis space of
explanations for the data generating process for the population, and each
`MODEL` is a candidate hypothesis. We start by creating only 1 model, which is
initialized __randomly__.

In [ ]:
%mml INITIALIZE 1 MODEL FOR "data";

### Visualizing a CrossCat model

As mentioned earlier, CrossCat learns the full joint distribution of all
variables in the population using divide-and-conquer:

- First, CrossCat partitions the variables into a set of _views_; all the
variables in a particular view are modeled jointly, and two variables in
different views are independent of one another.
- Second, within each view, CrossCat clusters the rows using a non-parametric
mixture model.

The name Cross-Categorization is derived from this two-step process: first
categorize the variables into views, and then categorize the rows into clusters
within each view of variables. It is important to note that two different views
A and B are likely to induce different clusterings of the rows.

To get a sense of CrossCat's hypothesis space, we can render the hypothesis
specified by a particular model using the `.render_crosscat [options]
<generator> <model_number>` plotting command. The `--subsample=50` option says
to only show a subsample of 50 rows in the rendering (even though the generator
is modeling all rows in the `data` population); finally `data
0` means to render the first (and only) model in the generator.

In [ ]:
%mml .render_crosscat \
    --subsample=50  --xticklabelsize=small --yticklabelsize=xx-small --progress=True --width=64 \
    data 0

__To view a full-size image of the rendering, either double click the image, or
right-click and select "Open image in new tab."__

Again, we emphasize that the CrossCat hypothesis shown above is __randomly__
initialized based on the two-step clustering process we have described. Each
block of columns shows a view of dependent variables. The clusters within a view
are demarcated using solid pink lines.
The color of a cell shows the magnitude of the data (normalized between 0 and 1,
where light indicates lower values and dark indices higher values).

### Using BQL to query CrossCat models

In the CrossCat rendering, each pair of variables is either in the same view
(and therefore probably dependent), or in different views (and therefore
independent). We can query the detected probable dependencies between all pairs
of variables using the `DEPENDENCE PROBABILITY` estimator in BQL. The next query
produces a heatmap of all pairs of dependencies. In the heatmap below, each row
and column is a variable, and the color of a cell is a value between 0 and 1
(lighter is nearer to 0, and darker is nearer to 1) indicating the amount of
evidence for a predictive relationship or dependency between these two
variables. Since we have initialized only 1 CrossCat model, each cell is exactly
either 0 (if those variables are in different views), or 1 (variables are in the
same view). Confirm that the blocks shown in the heatmap match up with the
blocks of variables from the rendering.

In [ ]:
%bql .interactive_heatmap ESTIMATE DEPENDENCE PROBABILITY FROM PAIRWISE VARIABLES OF data;

### Improving Crosscat hypotheses using MML `ANALYZE`

Now that we have initialized a CrossCat hypothesis and visualized its column and
row clusterings, it is time to improve our initial guess by exploring the
hypothesis space to find hypotheses that better explain the data. In particular,
our single CrossCat model has both spurious dependencies as well as
independencies between variables which we would expect to be depedenct (study
the heatmap and rendering, can you locate some of these pairs?).

We can improve the CrossCat model by using the MML `ANALYZE` command, which
takes the name of a generator, an amount of iterations or seconds, and optional
arguments. It then searches for improved hypotheses.

In [ ]:
%mml ANALYZE "data" FOR 50 ITERATIONS;

Let us look at the new CrossCat after running 200 steps of analysis. Study the
dependent variables. Can you identify a "theme" or "category" which summarizes
each view?

In [ ]:
%mml .render_crosscat \
    --subsample=50  --xticklabelsize=small --yticklabelsize=xx-small --progress=True --width=64 \
    data 0

We can again visualize the probability there exists a dependence, between all
pairs of variables, using BQL. How does this heatmap differ qualitatively from
the dependence probability heatmap we plotted prior to running `ANALYZE`?

In [ ]:
%bql .interactive_heatmap ESTIMATE DEPENDENCE PROBABILITY FROM PAIRWISE VARIABLES OF data;

### Initializing more CrossCat models

So far, we used `INITIALIZE 1 MODEL FOR data` to create a single model in
the ensemble. As a result, all of our heatmaps (such as a variable dependencies
and row similarities) had "sharp" values (either 1 or 0). Since CrossCat has a
very large hypothesis space, we can significantly improve modeling by creating
an ensemble of models, where each model searches the hypothesis space for
hypotheses that fit the data well. All queries in BQL will then become weighted
averages of the query results from each individual model in the ensemble.

The `%multiprocess on` magic activates multiprocessing BayesDB which allow us to
initialize, analyze and run queries on analyses using multiple cores on the host
machine.

In [ ]:
%multiprocess on

The following MML command ensures the generator will have a total of 32 models
in the ensemble (recall that we already initialized 1 model, so 31 new models
will be added to the ensemble).

In [ ]:
%mml INITIALIZE 10 MODELS IF NOT EXISTS FOR "data";

Again, we run analysis for a while.

In [ ]:
%mml ANALYZE "data" FOR 150 ITERATIONS (OPTIMIZED);

Let use produce some renderings of the models (here we choose 5, 7 and 15).
Where is there consensus among these three analyses? Where do they disagree?

In [ ]:
%mml .render_crosscat \
    --subsample=50  --xticklabelsize=small --yticklabelsize=x-small data 5
%mml .render_crosscat \
    --subsample=50  --xticklabelsize=small --yticklabelsize=x-small data 7
%mml .render_crosscat \
    --subsample=50  --xticklabelsize=small --yticklabelsize=x-small data 15

### Exploring probable dependencies between variables and comparing CrossCat
dependence probability to linear (Pearson R) correlation

As mentioned earlier, all BQL queries are aggregated across the 32 analyses in
the ensemble. We will create a table named `dependencies` which contains the
pairwise `DEPENDENCE PROBABILITY` values between the variables in the data. The
value of a cell (between 0 and 1) is the fraction of analyses in the ensemble
where those two variables are detected to be probably dependent (i.e. they are
in the same view).

In [ ]:
%%bql
CREATE TABLE dependencies AS
ESTIMATE
    DEPENDENCE PROBABILITY AS "depprob"
FROM PAIRWISE VARIABLES OF data;

Here are five random rows from the `dependencies` table.

In [ ]:
%bql SELECT * FROM "dependencies" ORDER BY RANDOM() LIMIT 5;

We again summarize the `dependencies` table using a heatmap. Study this
dependence heatmap, and compare it to the heatmap produced when there was only 1
model. Which common-sense dependencies were missed by the single model, but
identified by the ensemble as probably dependent?

In [ ]:
%bql .interactive_heatmap SELECT name0, name1, depprob FROM dependencies;

Let us compare dependence probabilities from CrossCat to linear (Pearson r)
correlation values, a very common technique for finding predictive
relationships. We can compute the Pearson R (and its p-value) in BayesDB using
the `CORRELATION` and `CORRELATION PVALUE` queries. The following cell creates a
table named `correlations`, which contains the R and p-value for all pairs of
variables.

In [ ]:
%%bql
CREATE TABLE "correlations" AS
ESTIMATE
    CORRELATION AS "correlation",
    CORRELATION PVALUE AS "pvalue"
FROM PAIRWISE VARIABLES OF "data"

Here are five random rows from the `correlations` table.

In [ ]:
%bql SELECT * FROM "correlations" ORDER BY RANDOM() LIMIT 5;

__Emphasis__: There is a signficiant difference between `DEPENDENCE
PROBABILITY`, `CORRELATION`, and `CORRELATION PVALUE`. We outline these
differences below, which will help us make comparisons between predictive
relationships detected by CrossCat versus Pearson correlation.

- `DEPENDENCE PROBABILITY`: Returns a value between [0,1] indicating the
__probability there exists__ a predictive relationship (statistical dependence)
between two variables.

- `CORRELATION`: Returns a value between [0,1] indicating the __strength__ of
the linear relationsip between two variables, where 0 means no linear
correlation, and 1 means perfect linear correlation.

- `CORRELATION PVALUE`: Returns a value between (0, 1) indicating the tail
probability of the observed correlation value between two variables, under the
null hypothesis that the two variables have zero correlation.

Based on these distinctions, there is no immediate way to numerically compare
`DEPENDENCE PROBABILITY` with `CORRELATION/CORRELATION PVALUE`. However, it is
possible to compare the inferences about predictive relationships that each
method gives rise to, which we do in the next section.

Let us first produce a heatmap of the raw correlation values. The following
query shows the raw correlation values (between 0 and 1) for all pairs of
variables where the p-value is less than 0.01 (note that we are not accounting
for multiple-testing using e.g. Bonferroni correction). Pairs of variables where
the p-value exceeds 0.01 (and thus the null hypothesis of independence cannot be
rejected) are shown in gray. The sparsity of the data makes it difficult to draw
inferences about many variables.

In [ ]:
%bql .interactive_heatmap SELECT name0, name1, "correlation" FROM "correlations" WHERE "pvalue" < 0.01

Explore the heatmap, and compare it to the heatmap from `DEPENDENCE
PROBABILITY`. The patterns of dependence relationships differ significantly,
how?

We can use BQL to find variables which CrossCat believes are probably dependent,
but correlation believes are independent (either the null hypothesis of
independence cannot be rejected, or the correlation value is significant and
near zero).

In [ ]:
%%sql
SELECT
    "name0",
    "name1",
    "dependencies"."depprob",
    "correlations"."correlation",
    "correlations"."pvalue"
FROM
    "dependencies"
    JOIN "correlations"
    USING ("name0", "name1")
WHERE
    -- CrossCat: probability dependent.
    "dependencies"."depprob" > 0.85
    AND (
    -- Correlation: cannot reject null hypothesis of independence.
    "correlations"."pvalue" > 0.05
    OR (
    -- Correlation: linear relationship is significant and near zero.
    "correlations"."pvalue" < 0.05 AND "correlations"."correlation" < 0.05))

We can use also BQL to find variables which CrossCat believes are probably
independent, but correlation believes are dependent (a statistically significant
non-zero correlation value, where we are using an R cutoff of 0.15). The
following query shows a list of such variables.

In [ ]:
%%sql
SELECT
    "name0",
    "name1",
    "dependencies"."depprob",
    "correlations"."correlation",
    "correlations"."pvalue"
FROM
    "dependencies"
    JOIN "correlations"
    USING ("name0", "name1")
WHERE
    -- CrossCat: high uncertainty about dependence probability.
    "dependencies"."depprob" < 0.05
    AND (
    -- Correlation: statistically significant dependence.
    "correlations"."pvalue" < 0.05 AND "correlations"."correlation" > 0.15)
LIMIT 10

Oftentimes, we notice that linear correlation is deceived into detecting a
dependency due to a single outlier in both cases. As a non-parametric mixture
model, CrossCat is more robust to outliers and irregular patterns in the data,
especially when there is insufficient evidence in the data to result in CrossCat
reporting probable dependencies (as is the case in the two scatter plots, with
only one data point deviating from the zero-dependence trend).

Bonferroni correction for multiple testing would perhaps render the p-values of
these correlations as statistically insignificant. However, Bonferroni is also
highly conservative, and will cause many common-sense relationships to be
insigificant as well under linear correlation. These design trade-offs are very
common in drawing inferences from frequentist methods such Pearson R.

Some next questions you might explore include:

- For which variables do CrossCat and linear correation agree about
dependencies?
- Which pairs of variables have the most uncertainty about their dependence
probability (a dependence probability value of 0.5 represents the most
uncertainy, or a light green color)?


### Exploring the clustering of the with respect to experimental outcomes

Recall that in addition to learning a clustering of variables, CrossCat
additionally learns a clustering of the rows within each view. These clusters
are separated using pink lines in the CrossCat rendering. We can use the
`SIMILARITY IN THE CONTEXT OF <variable>` query in BQL to study CrossCat's row
partition in the view of `<variable>`.

In the heatmaps below, each row and column is a row in the data table, and the value 
of a cell (between 0 and 1) indicates the probability that those two rows are
relevant for formulating predictions about each other.  We produces one heatmap
for each experimental outcome. Do these clusterings make sense?

In [ ]:
import matplotlib.pyplot as plt
for outcome in outcomes:
    fig, ax = plt.subplots()
    %bql .heatmap --label0=rowid --table=data ESTIMATE SIMILARITY IN THE CONTEXT OF {outcome}\
        FROM PAIRWISE data;

##### Interactive version

We can't plot all the interactive plots automatically because the javascript objects will overwrite each other.

In [ ]:
%bql .interactive_heatmap --label0=rowid --table=data \
    ESTIMATE SIMILARITY IN THE CONTEXT OF {outcomes[0]} FROM PAIRWISE data;